In [1]:
# Mountain Car- v0 solution using action-value neural network function approximator

In [2]:
import torch
import torch.nn as nn
from torchvision import transforms

# One hidden layer function approximator
class action_value_function(nn.Module):

    def __init__(self):

        super(action_value_function, self).__init__()
        self.block = nn.Sequential(
            torch.nn.Linear(2, 100),
            torch.nn.ReLU(),
            torch.nn.Linear(100, 50),
            torch.nn.ReLU(),
            torch.nn.Linear(50, 3),
            torch.nn.Softmax(),
        )

    def forward(self, input):
        return self.block(input)

In [3]:
# Instaciate action_value_function class
q_hat = action_value_function()

In [4]:
import gym
import numpy as np

done = False
LEARNING_RATE = 0.1
DISCOUNT = 0.95
EPISODES = 25000
SHOW_EVERY = 1000
BATCH_SIZE = 4

env = gym.make("MountainCar-v0")
env.reset()

DISCRETE_OS_SIZE = [20, 20] #* len(env.observation_space.high)
discrete_os_win_size = (env.observation_space.high - env.observation_space.low)/DISCRETE_OS_SIZE


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/home/ivlabs/.local/lib/python3.5/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [5]:
# Test number of actions and other parameters
print('observation_space:',env.observation_space.low,'to',env.observation_space.high,'| Number of action values:',env.action_space.n)

observation_space: [-1.2  -0.07] to [0.6  0.07] | Number of action values: 3


In [6]:
# Function for discretization of observation state space
def get_discrete_state(state):
    discrete_state = (state - env.observation_space.low)/discrete_os_win_size
    return tuple(discrete_state.astype(np.int))  # we use this tuple to look up the 3 Q values for the available actions in the q-table

# Sampled version of q value table
q_table = np.random.uniform(low=-2, high=0, size=(DISCRETE_OS_SIZE + [env.action_space.n]))

In [7]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(q_hat.parameters(), lr=0.0002, betas = (0.5,0.999))

In [8]:
env.action_space.sample()


0

In [17]:
avg_loss = 0
epsilon = 1

In [18]:
# Train agent
import random

for episode in range(EPISODES):
    state = torch.from_numpy(env.reset()).float()
#     torch.tensor(state)
    done = False
    
    if episode%SHOW_EVERY == 0:
        print('#'*50,'| RENDERING |','#'*49)
        RENDER = True
    else:
        RENDER = False

    running_loss = 0
    while not done:

        optimizer.zero_grad()
        
        action = torch.argmax(q_hat(state)).numpy()
        if random.random() < epsilon:
            action = env.action_space.sample()
        next_state, reward, done, _ = env.step(action)
        next_state = torch.from_numpy(next_state).float()
        
        if RENDER:
            env.render()

#         if not done:
 
#             max_future_q = max(q_hat(next_state))

        loss = criterion(reward + DISCOUNT * max(q_hat(next_state)), max(q_hat(state)))
#             print(loss)
        loss.backward()
        optimizer.step()
        # zero the parameter gradients
        

        # print statistics
        running_loss += loss.item()
    epsilon = epsilon / 2
        
    avg_loss += running_loss
    if episode%500 == 0:
        print('Episode:',episode,'| Average Loss:',avg_loss/500, '| Epsilon:',epsilon)
        avg_loss = 0

#        state = next_state

env.close()

################################################## | RENDERING | #################################################
Episode: 0 | Average Loss: 0.44099993705749513 | Epsilon: 0.5
Episode: 500 | Average Loss: 220.49996852874756 | Epsilon: 1.5274681817498023e-151
################################################## | RENDERING | #################################################
Episode: 1000 | Average Loss: 220.49996852874756 | Epsilon: 4.6663180925160944e-302
Episode: 1500 | Average Loss: 220.49996852874756 | Epsilon: 0.0
################################################## | RENDERING | #################################################
Episode: 2000 | Average Loss: 220.49996852874756 | Epsilon: 0.0
Episode: 2500 | Average Loss: 220.49996852874756 | Epsilon: 0.0
################################################## | RENDERING | #################################################
Episode: 3000 | Average Loss: 220.49996852874756 | Epsilon: 0.0


KeyboardInterrupt: 

In [11]:
epsilon

-580.3654400000001

In [ ]:
env.close()

In [ ]:
! pip install lib

In [ ]:
# coding: utf-8

# In[4]:

import gym
import itertools
import matplotlib
import numpy as np
import sys
import sklearn.pipeline
import sklearn.preprocessing

if "../" not in sys.path:
    sys.path.append("../") 

# from lib import plotting
from sklearn.linear_model import SGDRegressor
from sklearn.kernel_approximation import RBFSampler

matplotlib.style.use('ggplot')


# In[5]:


env = gym.envs.make("MountainCar-v0")


# In[9]:


# Feature Preprocessing: Normalize to zero mean and unit variance
# We use a few samples from the observation space to do this
observation_examples = np.array([env.observation_space.sample() for x in range(10000)])
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(observation_examples)


# In[7]:




# Used to convert a state to a featurized representation.
# We use RBF kernels with different variances to cover different parts of the space
featurizer = sklearn.pipeline.FeatureUnion([
        ("rbf1", RBFSampler(gamma=5.0, n_components=100)),
        ("rbf2", RBFSampler(gamma=2.0, n_components=100)),
        ("rbf3", RBFSampler(gamma=1.0, n_components=100)),
        ("rbf4", RBFSampler(gamma=0.5, n_components=100))
        ])
featurizer.fit(scaler.transform(observation_examples))


# In[119]:


class Estimator():
    """
    Value Function approximator. 
    """
    
    def __init__(self):
        # We create a separate model for each action in the environment's
        # action space. Alternatively we could somehow encode the action
        # into the features, but this way it's easier to code up.
        self.models = []
        for _ in range(env.action_space.n):
            model = SGDRegressor(learning_rate="constant")
            # We need to call partial_fit once to initialize the model
            # or we get a NotFittedError when trying to make a prediction
            # This is quite hacky.
            model.partial_fit([self.featurize_state(env.reset())], [0])
            self.models.append(model)
    
    def featurize_state(self, state):
        """
        Returns the featurized representation for a state.
        """
        scaled = scaler.transform([state])
        featurized = featurizer.transform(scaled)
        return featurized[0]
    
    def predict(self, s, a=None):
        """
        Makes value function predictions.
        
        Args:
            s: state to make a prediction for
            a: (Optional) action to make a prediction for
            
        Returns
            If an action a is given this returns a single number as the prediction.
            If no action is given this returns a vector or predictions for all actions
            in the environment where pred[i] is the prediction for action i.
            
        """
                
        # TODO: Implement this!
        if a is not None:
            prediction = self.models[a].predict([self.featurize_state(s)])
            return prediction[0]
        
        else:
            predictions = np.array([self.models[i].predict([self.featurize_state(s)]) for i in range(env.action_space.n)])
            return predictions.reshape(-1)
            
    def update(self, s, a, y):
        """
        Updates the estimator parameters for a given state and action towards
        the target y.
        """
        # TODO: Implement this!
        self.models[a].partial_fit([self.featurize_state(s)], [y])


# In[120]:


def make_epsilon_greedy_policy(estimator, epsilon, nA):
    """
    Creates an epsilon-greedy policy based on a given Q-function approximator and epsilon.
    
    Args:
        estimator: An estimator that returns q values for a given state
        epsilon: The probability to select a random action . float between 0 and 1.
        nA: Number of actions in the environment.
    
    Returns:
        A function that takes the observation as an argument and returns
        the probabilities for each action in the form of a numpy array of length nA.
    
    """
    def policy_fn(observation):
        A = np.ones(nA, dtype=float) * epsilon / nA
        q_values = estimator.predict(observation)
        best_action = np.argmax(q_values)
        A[best_action] += (1.0 - epsilon)
        return A
    return policy_fn


# In[121]:


def q_learning(env, estimator, num_episodes, discount_factor=1.0, epsilon=0.1, epsilon_decay=1.0):
    """
    Q-Learning algorithm for off-policy TD control using Function Approximation.
    Finds the optimal greedy policy while following an epsilon-greedy policy.
    
    Args:
        env: OpenAI environment.
        estimator: Action-Value function estimator
        num_episodes: Number of episodes to run for.
        discount_factor: Gamma discount factor.
        epsilon: Chance the sample a random action. Float betwen 0 and 1.
        epsilon_decay: Each episode, epsilon is decayed by this factor
    
    Returns:
        An EpisodeStats object with two numpy arrays for episode_lengths and episode_rewards.
    """
    
    nA = env.action_space.n
    
    # Keeps track of useful statistics
    stats = plotting.EpisodeStats(
        episode_lengths=np.zeros(num_episodes),
        episode_rewards=np.zeros(num_episodes))    
    
    for i_episode in range(num_episodes):
        
        # The policy we're following
        policy = make_epsilon_greedy_policy(
            estimator, epsilon * epsilon_decay**i_episode, nA)
        
        # Print out which episode we're on, useful for debugging.
        # Also print reward for last episode
        last_reward = stats.episode_rewards[i_episode - 1]
        print("\rEpisode {}/{} ({})".format(i_episode + 1, num_episodes, last_reward))
        sys.stdout.flush()
        
        # TODO: Implement this!
        state = env.reset()
        
        for t in itertools.count():
            
            # sample the action from the epsilon greedy policy
            action = np.random.choice(nA, p=policy(state))
            
            # Perform the action -> Get the reward and observe the next state
            new_state, reward, terminated, _ = env.step(action)
            new_action = np.random.choice(nA, p=policy(new_state))
                        
            stats.episode_rewards[i_episode] += reward
            stats.episode_lengths[i_episode] = t
            
            q_values_new_state = estimator.predict(new_state)

            # value that we should have got
            # The Q-learning target policy is a greedy one, hence the `max`
            td_target = reward + discount_factor * np.max(q_values_new_state)
            estimator.update(state, action, td_target)            
            
            # update current state
            state = new_state
            
            if terminated:
                break
    
    return stats


# In[122]:


estimator = Estimator()


# In[123]:


# Note: For the Mountain Car we don't actually need an epsilon > 0.0
# because our initial estimate for all states is too "optimistic" which leads
# to the exploration of all states.
stats = q_learning(env, estimator, 100, epsilon=0.0)


# In[124]:


plotting.plot_cost_to_go_mountain_car(env, estimator)
plotting.plot_episode_stats(stats, smoothing_window=25)


In [ ]:
! conda install matplotlib
